In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25519239
paper_name = 'aiyar_steinmetz_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [31]:
original_data = pd.read_csv('raw_data/ncomms6585-s3.txt', sep='\t')

In [32]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 6268 x 10


In [33]:
original_data.head()

,Strain,Gene,z_score,Fold_change,Description,GO_function,GO_process,GO_component,Essential,Zygosity
0,YAL001C::chr1_1,TFC3,1.836880,0.237992,Largest of six subunits of the RNA polymerase ...,DNA binding (IEA); RNA polymerase III transcri...,"regulation of transcription, DNA-dependent (IE...","mitochondrion (IDA, IEA); nucleus (IEA); trans...",yes,het
1,YAL002W::chr1_1,VPS8,2.867630,0.425052,Membrane-associated protein that interacts wit...,metal ion binding (IEA); molecular_function (N...,"late endosome to vacuole transport (IGI, IMP);...",CORVET complex (IDA); Golgi apparatus (IEA); m...,no,hom
2,YAL003W::chr1_1,EFB1,-3.019740,-0.257603,Translation elongation factor 1 beta; stimulat...,"translation elongation factor activity (TAS, IEA)",translation (IEA); translational elongation (T...,eukaryotic translation elongation factor 1 com...,yes,het
3,YAL004W::chr1_1,YAL004W,-0.978514,-0.105441,Dubious open reading frame unlikely to encode ...,NaN,NaN,NaN,no,hom
4,YAL005C::chr1_1,SSA1,1.537400,0.135975,ATPase involved in protein folding and nuclear...,ATP binding (IEA); ATPase activity (IDA); nucl...,SRP-dependent cotranslational protein targetin...,cell wall (IEA); chaperonin-containing T-compl...,no,hom


In [34]:
original_data['orf'] = original_data['Strain'].apply(lambda x: x.split(':')[0])

In [35]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [36]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [37]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Strain, Gene, z_score, Fold_change, Description, GO_function, GO_process, GO_component, Essential, Zygosity, orf]
Index: []


In [38]:
# Flip the sign since z-scores were calculated as (u_ctrl-u_treat)/s_ctrl
original_data['data'] = -original_data['z_score']

In [39]:
original_data.set_index('orf', inplace=True)

In [40]:
original_data1 = original_data.loc[original_data['Zygosity']=='hom',['data']].copy()
original_data2 = original_data.loc[original_data['Zygosity']=='het',['data']].copy()

In [41]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [42]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [43]:
original_data.shape

(6017, 2)

In [44]:
original_data.head()

,data_1,data_2
orf,,
YAL001C,NaN,-1.83688
YAL002W,-2.867630,NaN
YAL003W,NaN,3.01974
YAL004W,0.978514,NaN
YAL005C,-1.537400,NaN


# Prepare the final dataset

In [45]:
data = original_data.copy()

In [46]:
dataset_ids = [16251, 16252]
datasets = datasets.reindex(index=dataset_ids)

In [47]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [48]:
data.head()

dataset_id,16251,16252
data_type,value,value
orf,,
YAL001C,NaN,-1.83688
YAL002W,-2.867630,NaN
YAL003W,NaN,3.01974
YAL004W,0.978514,NaN
YAL005C,-1.537400,NaN


## Subset to the genes currently in SGD

In [49]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [50]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16251,16252
,data_type,value,value
gene_id,orf,,
1,YAL001C,NaN,-1.83688
2,YAL002W,-2.867630,NaN
3,YAL003W,NaN,3.01974
1863,YAL004W,0.978514,NaN
4,YAL005C,-1.537400,NaN


# Normalize

In [51]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [52]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [53]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16251,16252,16251,16252
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1,YAL001C,NaN,-1.83688,NaN,-0.606570
2,YAL002W,-2.867630,NaN,-2.907873,NaN
3,YAL003W,NaN,3.01974,NaN,1.098177
1863,YAL004W,0.978514,NaN,0.794774,NaN
4,YAL005C,-1.537400,NaN,-1.627273,NaN


# Print out

In [54]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [55]:
from IO.save_data_to_db3 import *

In [56]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 25519239...
Inserting the new data...


100%|██████████| 2/2 [00:17<00:00,  8.83s/it]

Updating the data_modified_on field...
